In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
use_cuda = True       
        

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
train_df = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/train.csv')
test_df = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/test.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df['sentiment'].value_counts().plot(kind='bar')

In [ ]:
train_df.count()

In [ ]:
train_df.dropna(inplace=True)

In [ ]:
train_df.count()

In [ ]:
##As we following Question Answer approach. We need to convert our train / test data into Q & A accepting format.

In [ ]:
"""Given a question and context, the model is trained to find the answer spans in the context.

Therefore, used sentiment as question, text as context, selected_text as answer.

    Question: sentiment
    Context: text
    Answer: selected_text"""


In [ ]:
"""   Example-format:

train_data = [
    {
        'context': "This tweet sentiment extraction challenge is great",
        'qas': [
            {
                'id': "00001",
                'question': "positive",
                'answers': [
                    {
                        'text': "is great",
                        'answer_start': 43
                    }
                ]
            }
            """

In [ ]:
"""
Data format

For question answering tasks, the input data can be in JSON files or in a Python list of dicts in the correct format.

The file should contain a single list of dictionaries. A dictionary represents a single context and its associated questions.

Each such dictionary contains two attributes, the "context" and "qas".

    context: The paragraph or text from which the question is asked.
    qas: A list of questions and answers.

Questions and answers are represented as dictionaries. Each dictionary in qas has the following format.

    id: (string) A unique ID for the question. Should be unique across the entire dataset.
    question: (string) A question.
    is_impossible: (bool) Indicates whether the question can be answered correctly from the context.
    answers: (list) The list of correct answers to the question.

A single answer is represented by a dictionary with the following attributes.

    answer: (string) The answer to the question. Must be a substring of the context.
    answer_start: (int) Starting index of the answer in the context.

"""

In [ ]:
build_train_data(train_df)[1]

In [ ]:
def build_train_data(df):
    train_data = []
    for index in df.index:
        context = df.loc[index,'text']
        idx = df.loc[index,'textID']
        que = 'what is '+df.loc[index,'sentiment']
        ans = df.loc[index,'selected_text']
        start_pos = df.loc[index,'text'].find(df.loc[index,'selected_text'])# first occurence of specified value
        
        qas = []
        answers = []
        ans_dict = {}
        qa_dict = {}
        ans_dict = {'text': ans,'answer_start':start_pos}
        answers.append(ans_dict)
        qa_dict = {'id':idx,'question': que,'is_impossible':False,'answers':answers}
        qas.append(qa_dict)
        cont_dict ={'context':context,'qas':qas}
        train_data.append(cont_dict)
    return train_data

In [ ]:
train_df[0:5]

In [ ]:
build_train_data(train_df)[1]

In [ ]:
test_df[:5]

In [ ]:
build_test_data(test_df)[1]

In [ ]:
def build_test_data(df):
    test_data = []
    for index in df.index:
        context = df.loc[index,'text']
        idx = df.loc[index,'textID']
        que = 'what is '+df.loc[index,'sentiment']

        
        qas = []
        answers = []
        ans_dict = {}
        qa_dict = {}
        ans_dict = {'text': '__NONE__','answer_start':1000000}
        answers.append(ans_dict)
        qa_dict = {'id':idx,'question': que,'is_impossible':False,'answers':answers}
        qas.append(qa_dict)
        cont_dict ={'context':context,'qas':qas}
        test_data.append(cont_dict)
    return test_data

In [ ]:
build_test_data(test_df)[1]

In [ ]:
distil_accept_train = build_train_data(train_df)

In [ ]:
distil_accept_train[0:1]

In [ ]:
distil_accept_test = build_test_data(test_df)

In [ ]:
distil_accept_test[0:1]

In [ ]:
####### install packages

In [ ]:
!pip install '/kaggle/input/simple-transformers-pypi/seqeval-0.0.12-py3-none-any.whl' -q

In [ ]:
!pip install '/kaggle/input/simple-transformers-pypi/simpletransformers-0.22.1-py3-none-any.whl' -q

In [ ]:
####### Building Question Answering System

In [ ]:
from simpletransformers.question_answering import QuestionAnsweringModel

In [ ]:
model_path = '/kaggle/input/transformers-pretrained-distilbert/distilbert-base-uncased-distilled-squad'

In [ ]:

%%time

model = QuestionAnsweringModel('distilbert',model_path,args={'reprocess_input_data': True,
                                     'overwrite_output_dir': True,                    
                                     'learning_rate': 5e-5,
                                     'num_train_epochs': 3,
                                     'max_seq_length': 192,
                                     'doc_stride': 64,
                                     'fp16': False,                      
                                    },use_cuda=True)



In [ ]:
%%time
model.train_model(distil_accept_train)

In [ ]:
pred =  model.predict(distil_accept_test)

In [ ]:
pred[0:8]

In [ ]:
sub_df = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/sample_submission.csv')

In [ ]:
submit_df = pd.DataFrame.from_dict(pred)
sub_df['selected_text'] = submit_df['answer']
sub_df.to_csv('submission.csv',index=False)
print('successfully submit')